# Phase 1: Exploratory Data Analysis

This notebook provides a comprehensive exploratory data analysis of the Telco Customer Churn dataset, focusing on:

1. **Churn Ratios**: Overall churn rate and churn rates by various customer segments
2. **Categorical Breakdowns**: Analysis of how different categorical features relate to churn
3. **Tenure Trends**: Distribution of customer tenure and its relationship with churn
4. **Charges Trends**: Analysis of MonthlyCharges and TotalCharges in relation to churn

## Objectives
- Understand the dataset structure and quality
- Identify key factors associated with customer churn
- Generate insights to inform feature engineering and modeling strategies


In [ ]:
import sys
from pathlib import Path

# Add project root to path
PROJECT_ROOT = Path().resolve().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set style
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)

# Load data
raw_path = PROJECT_ROOT / "data" / "raw" / "telco_data_28_11_2025.csv"
df = pd.read_csv(raw_path)

# Clean data (matching ingestion pipeline)
df.columns = [c.strip() for c in df.columns]
df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors="coerce")
df["SeniorCitizen"] = df["SeniorCitizen"].astype("Int64")
df = df.dropna(subset=["customerID"])
df["customerID"] = df["customerID"].str.strip()

print(f"Dataset shape: {df.shape}")
print(f"\nColumns: {list(df.columns)}")
df.head()


In [ ]:
# Overall churn ratio
churn_counts = df["Churn"].value_counts()
churn_pct = df["Churn"].value_counts(normalize=True) * 100

print("=" * 50)
print("OVERALL CHURN RATIO")
print("=" * 50)
for label in ["Yes", "No"]:
    print(f"{label:10s}: {churn_counts[label]:5d} ({churn_pct[label]:5.2f}%)")

# Visualize overall churn distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Count plot
churn_counts.plot(kind="bar", ax=axes[0], color=["#2ecc71", "#e74c3c"])
axes[0].set_title("Churn Distribution (Counts)", fontsize=14, fontweight="bold")
axes[0].set_xlabel("Churn")
axes[0].set_ylabel("Count")
axes[0].set_xticklabels(["No", "Yes"], rotation=0)
for i, v in enumerate(churn_counts.values):
    axes[0].text(i, v + 50, str(v), ha="center", va="bottom", fontweight="bold")

# Pie chart
churn_pct.plot(kind="pie", ax=axes[1], autopct="%1.1f%%", colors=["#2ecc71", "#e74c3c"])
axes[1].set_title("Churn Distribution (Percentages)", fontsize=14, fontweight="bold")
axes[1].set_ylabel("")

plt.tight_layout()
plt.show()

print(f"\n⚠️  Class Imbalance: {churn_pct['Yes']:.1f}% churn rate indicates imbalanced dataset")
print("   Consider using class weights or resampling techniques in modeling.")


In [ ]:
# Churn ratios by key categorical features
categorical_features = [
    "gender",
    "SeniorCitizen",
    "Partner",
    "Dependents",
    "PhoneService",
    "InternetService",
    "Contract",
    "PaperlessBilling",
    "PaymentMethod",
]

print("=" * 80)
print("CHURN RATIOS BY CATEGORICAL FEATURES")
print("=" * 80)

churn_by_cat = {}
for feature in categorical_features:
    if feature == "SeniorCitizen":
        # Convert to string for consistency
        df_temp = df.copy()
        df_temp[feature] = df_temp[feature].astype(str).replace({"0": "No", "1": "Yes"})
    else:
        df_temp = df
    
    crosstab = pd.crosstab(df_temp[feature], df["Churn"], normalize="index") * 100
    churn_by_cat[feature] = crosstab
    
    print(f"\n{feature.upper()}:")
    print(crosstab.round(2))
    print(f"  Churn Rate (Yes): {crosstab['Yes'].mean():.2f}%")

# Visualize churn rates by key features
fig, axes = plt.subplots(3, 3, figsize=(18, 15))
axes = axes.flatten()

for idx, feature in enumerate(categorical_features[:9]):
    if feature == "SeniorCitizen":
        df_temp = df.copy()
        df_temp[feature] = df_temp[feature].astype(str).replace({"0": "No", "1": "Yes"})
    else:
        df_temp = df
    
    crosstab = pd.crosstab(df_temp[feature], df["Churn"])
    crosstab_pct = pd.crosstab(df_temp[feature], df["Churn"], normalize="index") * 100
    
    crosstab_pct.plot(kind="bar", ax=axes[idx], color=["#2ecc71", "#e74c3c"], stacked=True)
    axes[idx].set_title(f"Churn by {feature}", fontweight="bold")
    axes[idx].set_xlabel(feature)
    axes[idx].set_ylabel("Percentage (%)")
    axes[idx].legend(title="Churn", labels=["No", "Yes"])
    axes[idx].set_xticklabels(axes[idx].get_xticklabels(), rotation=45, ha="right")

plt.tight_layout()
plt.show()


## 2. Categorical Breakdowns

Detailed analysis of how categorical features relate to churn, including service combinations.


In [ ]:
# Service combinations analysis
service_features = [
    "OnlineSecurity",
    "OnlineBackup",
    "DeviceProtection",
    "TechSupport",
    "StreamingTV",
    "StreamingMovies",
]

print("=" * 80)
print("CHURN RATIOS BY SERVICE FEATURES")
print("=" * 80)

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

for idx, feature in enumerate(service_features):
    crosstab_pct = pd.crosstab(df[feature], df["Churn"], normalize="index") * 100
    
    crosstab_pct.plot(kind="bar", ax=axes[idx], color=["#2ecc71", "#e74c3c"], stacked=True)
    axes[idx].set_title(f"Churn by {feature}", fontweight="bold")
    axes[idx].set_xlabel(feature)
    axes[idx].set_ylabel("Percentage (%)")
    axes[idx].legend(title="Churn", labels=["No", "Yes"])
    axes[idx].set_xticklabels(axes[idx].get_xticklabels(), rotation=45, ha="right")
    
    # Print summary
    print(f"\n{feature}:")
    print(f"  Churn Rate by Category:")
    for category in crosstab_pct.index:
        churn_rate = crosstab_pct.loc[category, "Yes"]
        print(f"    {category:20s}: {churn_rate:5.2f}%")

plt.tight_layout()
plt.show()


In [ ]:
# Contract and Payment Method - Key drivers
print("=" * 80)
print("KEY INSIGHTS: CONTRACT & PAYMENT METHOD")
print("=" * 80)

# Contract analysis
contract_churn = pd.crosstab(df["Contract"], df["Churn"], normalize="index") * 100
print("\nContract Type Churn Rates:")
print(contract_churn.round(2))

# Payment Method analysis
payment_churn = pd.crosstab(df["PaymentMethod"], df["Churn"], normalize="index") * 100
print("\nPayment Method Churn Rates:")
print(payment_churn.round(2))

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

contract_churn.plot(kind="bar", ax=axes[0], color=["#2ecc71", "#e74c3c"], stacked=True)
axes[0].set_title("Churn by Contract Type", fontweight="bold", fontsize=14)
axes[0].set_xlabel("Contract Type")
axes[0].set_ylabel("Percentage (%)")
axes[0].legend(title="Churn", labels=["No", "Yes"])
axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=45, ha="right")

payment_churn.plot(kind="bar", ax=axes[1], color=["#2ecc71", "#e74c3c"], stacked=True)
axes[1].set_title("Churn by Payment Method", fontweight="bold", fontsize=14)
axes[1].set_xlabel("Payment Method")
axes[1].set_ylabel("Percentage (%)")
axes[1].legend(title="Churn", labels=["No", "Yes"])
axes[1].set_xticklabels(axes[1].get_xticklabels(), rotation=45, ha="right")

plt.tight_layout()
plt.show()

print("\n💡 Key Finding: Month-to-month contracts and Electronic check payments show highest churn rates!")


## 3. Tenure Trends

Analysis of customer tenure distribution and its relationship with churn.


In [ ]:
# Tenure distribution
print("=" * 80)
print("TENURE ANALYSIS")
print("=" * 80)

print(f"\nTenure Statistics:")
print(df["tenure"].describe())

# Tenure distribution by churn
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Overall tenure distribution
axes[0, 0].hist(df["tenure"], bins=50, color="#3498db", edgecolor="black", alpha=0.7)
axes[0, 0].set_title("Overall Tenure Distribution", fontweight="bold", fontsize=14)
axes[0, 0].set_xlabel("Tenure (months)")
axes[0, 0].set_ylabel("Frequency")
axes[0, 0].axvline(df["tenure"].mean(), color="red", linestyle="--", linewidth=2, label=f"Mean: {df['tenure'].mean():.1f}")
axes[0, 0].legend()

# Tenure by churn status
for churn_status in ["No", "Yes"]:
    tenure_data = df[df["Churn"] == churn_status]["tenure"]
    axes[0, 1].hist(tenure_data, bins=50, alpha=0.6, label=f"Churn: {churn_status}", edgecolor="black")
axes[0, 1].set_title("Tenure Distribution by Churn Status", fontweight="bold", fontsize=14)
axes[0, 1].set_xlabel("Tenure (months)")
axes[0, 1].set_ylabel("Frequency")
axes[0, 1].legend()

# Box plot: Tenure by churn
df.boxplot(column="tenure", by="Churn", ax=axes[1, 0])
axes[1, 0].set_title("Tenure Distribution by Churn", fontweight="bold", fontsize=14)
axes[1, 0].set_xlabel("Churn")
axes[1, 0].set_ylabel("Tenure (months)")
axes[1, 0].set_xticklabels(["No", "Yes"])

# Tenure buckets analysis
df["tenure_bucket"] = pd.cut(
    df["tenure"],
    bins=[0, 12, 24, 36, 48, 60, 72, 100],
    labels=["0-12", "13-24", "25-36", "37-48", "49-60", "61-72", "73+"]
)
tenure_bucket_churn = pd.crosstab(df["tenure_bucket"], df["Churn"], normalize="index") * 100
tenure_bucket_churn.plot(kind="bar", ax=axes[1, 1], color=["#2ecc71", "#e74c3c"], stacked=True)
axes[1, 1].set_title("Churn Rate by Tenure Buckets", fontweight="bold", fontsize=14)
axes[1, 1].set_xlabel("Tenure (months)")
axes[1, 1].set_ylabel("Percentage (%)")
axes[1, 1].legend(title="Churn", labels=["No", "Yes"])
axes[1, 1].set_xticklabels(axes[1, 1].get_xticklabels(), rotation=45, ha="right")

plt.tight_layout()
plt.show()

# Statistical summary
print("\nTenure Statistics by Churn Status:")
print(df.groupby("Churn")["tenure"].describe())

print("\n💡 Key Finding: Customers with shorter tenure (< 12 months) have significantly higher churn rates!")


In [ ]:
# Charges analysis
print("=" * 80)
print("CHARGES ANALYSIS")
print("=" * 80)

print("\nMonthlyCharges Statistics:")
print(df["MonthlyCharges"].describe())

print("\nTotalCharges Statistics (excluding missing):")
print(df["TotalCharges"].dropna().describe())

# Charges distribution by churn
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# MonthlyCharges distribution
axes[0, 0].hist(df["MonthlyCharges"], bins=50, color="#3498db", edgecolor="black", alpha=0.7)
axes[0, 0].set_title("Overall MonthlyCharges Distribution", fontweight="bold", fontsize=14)
axes[0, 0].set_xlabel("Monthly Charges ($)")
axes[0, 0].set_ylabel("Frequency")
axes[0, 0].axvline(df["MonthlyCharges"].mean(), color="red", linestyle="--", linewidth=2, 
                   label=f"Mean: ${df['MonthlyCharges'].mean():.2f}")
axes[0, 0].legend()

# MonthlyCharges by churn
for churn_status in ["No", "Yes"]:
    charges_data = df[df["Churn"] == churn_status]["MonthlyCharges"]
    axes[0, 1].hist(charges_data, bins=50, alpha=0.6, label=f"Churn: {churn_status}", edgecolor="black")
axes[0, 1].set_title("MonthlyCharges Distribution by Churn", fontweight="bold", fontsize=14)
axes[0, 1].set_xlabel("Monthly Charges ($)")
axes[0, 1].set_ylabel("Frequency")
axes[0, 1].legend()

# Box plot: MonthlyCharges by churn
df.boxplot(column="MonthlyCharges", by="Churn", ax=axes[0, 2])
axes[0, 2].set_title("MonthlyCharges by Churn", fontweight="bold", fontsize=14)
axes[0, 2].set_xlabel("Churn")
axes[0, 2].set_ylabel("Monthly Charges ($)")
axes[0, 2].set_xticklabels(["No", "Yes"])

# TotalCharges distribution (excluding missing)
df_clean = df.dropna(subset=["TotalCharges"])
axes[1, 0].hist(df_clean["TotalCharges"], bins=50, color="#3498db", edgecolor="black", alpha=0.7)
axes[1, 0].set_title("Overall TotalCharges Distribution", fontweight="bold", fontsize=14)
axes[1, 0].set_xlabel("Total Charges ($)")
axes[1, 0].set_ylabel("Frequency")
axes[1, 0].axvline(df_clean["TotalCharges"].mean(), color="red", linestyle="--", linewidth=2,
                   label=f"Mean: ${df_clean['TotalCharges'].mean():.2f}")
axes[1, 0].legend()

# TotalCharges by churn
for churn_status in ["No", "Yes"]:
    charges_data = df_clean[df_clean["Churn"] == churn_status]["TotalCharges"]
    axes[1, 1].hist(charges_data, bins=50, alpha=0.6, label=f"Churn: {churn_status}", edgecolor="black")
axes[1, 1].set_title("TotalCharges Distribution by Churn", fontweight="bold", fontsize=14)
axes[1, 1].set_xlabel("Total Charges ($)")
axes[1, 1].set_ylabel("Frequency")
axes[1, 1].legend()

# Scatter: MonthlyCharges vs TotalCharges colored by churn
scatter = axes[1, 2].scatter(
    df_clean["MonthlyCharges"],
    df_clean["TotalCharges"],
    c=df_clean["Churn"].map({"Yes": 1, "No": 0}),
    cmap="RdYlGn",
    alpha=0.5,
    s=20
)
axes[1, 2].set_title("MonthlyCharges vs TotalCharges", fontweight="bold", fontsize=14)
axes[1, 2].set_xlabel("Monthly Charges ($)")
axes[1, 2].set_ylabel("Total Charges ($)")
plt.colorbar(scatter, ax=axes[1, 2], label="Churn (1=Yes, 0=No)")

plt.tight_layout()
plt.show()

# Statistical summary
print("\nMonthlyCharges Statistics by Churn Status:")
print(df.groupby("Churn")["MonthlyCharges"].describe())

print("\nTotalCharges Statistics by Churn Status:")
print(df_clean.groupby("Churn")["TotalCharges"].describe())

print("\n💡 Key Finding: Customers with higher MonthlyCharges tend to churn more, but those with higher TotalCharges (longer tenure) churn less!")


In [ ]:
# Charges buckets analysis
df["monthly_charges_bucket"] = pd.cut(
    df["MonthlyCharges"],
    bins=[0, 35, 50, 65, 80, 100, 120],
    labels=["0-35", "36-50", "51-65", "66-80", "81-100", "101-120"]
)

df_clean["total_charges_bucket"] = pd.cut(
    df_clean["TotalCharges"],
    bins=[0, 500, 1000, 2000, 3000, 5000, 10000],
    labels=["0-500", "501-1000", "1001-2000", "2001-3000", "3001-5000", "5001+"]
)

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# MonthlyCharges buckets
monthly_bucket_churn = pd.crosstab(df["monthly_charges_bucket"], df["Churn"], normalize="index") * 100
monthly_bucket_churn.plot(kind="bar", ax=axes[0], color=["#2ecc71", "#e74c3c"], stacked=True)
axes[0].set_title("Churn Rate by MonthlyCharges Buckets", fontweight="bold", fontsize=14)
axes[0].set_xlabel("Monthly Charges ($)")
axes[0].set_ylabel("Percentage (%)")
axes[0].legend(title="Churn", labels=["No", "Yes"])
axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=45, ha="right")

# TotalCharges buckets
total_bucket_churn = pd.crosstab(df_clean["total_charges_bucket"], df_clean["Churn"], normalize="index") * 100
total_bucket_churn.plot(kind="bar", ax=axes[1], color=["#2ecc71", "#e74c3c"], stacked=True)
axes[1].set_title("Churn Rate by TotalCharges Buckets", fontweight="bold", fontsize=14)
axes[1].set_xlabel("Total Charges ($)")
axes[1].set_ylabel("Percentage (%)")
axes[1].legend(title="Churn", labels=["No", "Yes"])
axes[1].set_xticklabels(axes[1].get_xticklabels(), rotation=45, ha="right")

plt.tight_layout()
plt.show()

print("\nMonthlyCharges Buckets Churn Rates:")
print(monthly_bucket_churn.round(2))

print("\nTotalCharges Buckets Churn Rates:")
print(total_bucket_churn.round(2))


## 5. Key Insights Summary

### Top Churn Risk Factors:
1. **Contract Type**: Month-to-month contracts have the highest churn rate
2. **Payment Method**: Electronic check payments show elevated churn
3. **Tenure**: New customers (< 12 months) are at highest risk
4. **Monthly Charges**: Higher monthly charges correlate with increased churn
5. **Services**: Customers without add-on services (OnlineSecurity, TechSupport) churn more

### Recommendations for Feature Engineering:
- Create tenure buckets (0-12, 13-24, 25-36, etc.)
- Create charges ratio features (MonthlyCharges/TotalCharges)
- Create service count features (number of add-on services)
- Create contract-payment interaction features
- Consider creating customer lifetime value proxy (TotalCharges / tenure)


In [ ]:
# Correlation analysis
print("=" * 80)
print("CORRELATION ANALYSIS")
print("=" * 80)

# Prepare numeric features
numeric_df = df[["SeniorCitizen", "tenure", "MonthlyCharges", "TotalCharges"]].copy()
numeric_df["Churn_Numeric"] = df["Churn"].map({"Yes": 1, "No": 0})

correlation_matrix = numeric_df.corr()
print("\nCorrelation with Churn (Numeric):")
print(correlation_matrix["Churn_Numeric"].sort_values(ascending=False))

# Visualize correlation matrix
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap="coolwarm", center=0,
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title("Correlation Matrix: Numeric Features", fontweight="bold", fontsize=14, pad=20)
plt.tight_layout()
plt.show()

print("\n✅ EDA Complete! Ready for feature engineering and modeling.")
